In [1]:
import random
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import copy
import time

# Import both standard tqdm and our smart tqdm
from tqdm import tqdm  # Standard tqdm for comparison
from tqdmpp import SmartBar, accuracy_based_selector, loss_based_selector

def set_seed(seed=116):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  

set_seed(116)
print("Classification Demo with VGG16 + Smart TQDM++")
print("="*60)

Classification Demo with VGG16 + Smart TQDM++


In [2]:
class VGG16Classifier(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Classifier, self).__init__()
        
        # Load pre-trained VGG16
        self.vgg16 = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')
        
        # Freeze early layers (optional - for faster training)
        for param in self.vgg16.features[:20].parameters():
            param.requires_grad = False
        
        # Replace the classifier for our 3-class problem
        self.vgg16.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
    
    def forward(self, x):
        return self.vgg16(x)

# Create model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📱 Using device: {device}")

model = VGG16Classifier(num_classes=3)
model.to(device)

print("VGG16-based Classifier Created!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

📱 Using device: cpu
VGG16-based Classifier Created!
Total parameters: 121,677,635
Trainable parameters: 116,402,179


In [3]:
# Data loading and preprocessing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 standard input size
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

# Use local data directory
train_dir = "data/Dog_heart/Train"

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Smaller batch size for VGG16

print(f"Data loaded from: {train_dir}")
print(f"Number of training samples: {len(train_dataset)}")
print(f"Classes: {train_dataset.classes}")
print(f"Batch size: 16")
print(f"Number of batches: {len(train_loader)}")

Data loaded from: data/Dog_heart/Train
Number of training samples: 1400
Classes: ['Large', 'Normal', 'Small']
Batch size: 16
Number of batches: 88


In [4]:

# Training function with different progress bar modes
def train_with_progress_bar(model, train_loader, criterion, optimizer, num_epochs, progress_bar_type="default"):
    """
    Train model with specified progress bar type
    
    Args:
        progress_bar_type: "default", "accuracy", "loss", "standard"
    """
    print(f"Training with {progress_bar_type.upper()} progress bar...")
    
    model.train()
    history = {'train_loss': [], 'train_acc': []}
    
    for epoch in range(num_epochs):
        train_loss, correct, total = 0.0, 0, 0
        
        # Choose appropriate progress bar
        if progress_bar_type == "default":
            # Default SmartBar with intelligent emoji selection
            progress_bar = SmartBar(len(train_loader), 
                                  desc=f"Epoch {epoch+1}/{num_epochs} [Default Smart]", 
                                  show_emoji=True)
        elif progress_bar_type == "accuracy":
            # Accuracy-based emoji selection
            progress_bar = SmartBar(len(train_loader), 
                                  desc=f"Epoch {epoch+1}/{num_epochs} [Accuracy-based]", 
                                  show_emoji=True, 
                                  emoji_selector=accuracy_based_selector)
        elif progress_bar_type == "loss":
            # Loss-based emoji selection
            progress_bar = SmartBar(len(train_loader), 
                                  desc=f"Epoch {epoch+1}/{num_epochs} [Loss-based]", 
                                  show_emoji=True, 
                                  emoji_selector=loss_based_selector)
        else:  # standard
            # Standard tqdm for comparison
            progress_bar = tqdm(range(len(train_loader)), 
                              desc=f"Epoch {epoch+1}/{num_epochs} [Standard TQDM]")
        
        # Training loop
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Calculate metrics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Update progress bar
            current_loss = train_loss / (batch_idx + 1)
            current_acc = 100 * correct / total
            
            if progress_bar_type != "standard":
                # For SmartBar, update with metrics
                progress_bar.set_metrics(loss=f"{current_loss:.3f}", acc=f"{current_acc:.2f}")
                next(progress_bar)
            else:
                # For standard tqdm, just update position
                progress_bar.update(1)
                progress_bar.set_postfix(loss=f"{current_loss:.3f}", acc=f"{current_acc:.2f}%")
        
        # Close progress bar
        if hasattr(progress_bar, 'close'):
            progress_bar.close()
        
        # Calculate epoch metrics
        epoch_loss = train_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        print(f"Epoch {epoch+1} completed: Loss={epoch_loss:.4f}, Acc={epoch_acc:.2f}%")
        
        # Add small delay to see the emoji changes
        time.sleep(0.5)
    
    return history

# Setup training
criterion = nn.CrossEntropyLoss()
num_epochs = 2  # Reduced for demo purposes

print("Starting comprehensive training demo with 4 different progress bar modes!\n")

# Demo 1: Default SmartBar (intelligent emoji selection)
print("=" * 70)
print("DEMO 1: DEFAULT SMART TQDM++ (Intelligent Selection)")
print("=" * 70)
model1 = VGG16Classifier(num_classes=3).to(device)
optimizer1 = optim.Adam(model1.parameters(), lr=1e-4)
history1 = train_with_progress_bar(model1, train_loader, criterion, optimizer1, num_epochs, "default")

print("\n" + "=" * 70)
print("DEMO 2: ACCURACY-BASED SMART TQDM++")
print("=" * 70)
model2 = VGG16Classifier(num_classes=3).to(device)
optimizer2 = optim.Adam(model2.parameters(), lr=1e-4)
history2 = train_with_progress_bar(model2, train_loader, criterion, optimizer2, num_epochs, "accuracy")

print("\n" + "=" * 70)
print("DEMO 3: LOSS-BASED SMART TQDM++")
print("=" * 70)
model3 = VGG16Classifier(num_classes=3).to(device)
optimizer3 = optim.Adam(model3.parameters(), lr=1e-4)
history3 = train_with_progress_bar(model3, train_loader, criterion, optimizer3, num_epochs, "loss")

print("\n" + "=" * 70)
print("DEMO 4: STANDARD TQDM (Comparison)")
print("=" * 70)
model4 = VGG16Classifier(num_classes=3).to(device)
optimizer4 = optim.Adam(model4.parameters(), lr=1e-4)
history4 = train_with_progress_bar(model4, train_loader, criterion, optimizer4, num_epochs, "standard")

print("\nAll training demos completed!")
print("Notice how SmartBar provides intelligent emoji feedback based on performance trends!")

Starting comprehensive training demo with 4 different progress bar modes!

DEMO 1: DEFAULT SMART TQDM++ (Intelligent Selection)
Training with DEFAULT progress bar...

Terminal too narrow (80 chars). Bar will be minimal.
Epoch 1/2 [Defaul... ┃━━━━━━━━━━🚀┃ 88/88 100.0% [ 496.8s] [loss:0.807 acc:57.79]Epoch 1 completed: Loss=0.8068, Acc=57.79%

Terminal too narrow (80 chars). Bar will be minimal.
Epoch 2/2 [Defaul... ┃━━━━━━━━━━🚀┃ 88/88 100.0% [ 167.1s] [loss:0.664 acc:67.93]Epoch 2 completed: Loss=0.6645, Acc=67.93%

DEMO 2: ACCURACY-BASED SMART TQDM++
Training with ACCURACY progress bar...

Terminal too narrow (80 chars). Bar will be minimal.
Epoch 1/2 [Accura... ┃━━━━━━━━━━😅┃ 88/88 100.0% [ 180.3s] [loss:0.853 acc:53.07]Epoch 1 completed: Loss=0.8531, Acc=53.07%

Terminal too narrow (80 chars). Bar will be minimal.
Epoch 2/2 [Accura... ┃━━━━━━━━━━🚀┃ 88/88 100.0% [ 178.6s] [loss:0.659 acc:67.86]Epoch 2 completed: Loss=0.6588, Acc=67.86%

DEMO 3: LOSS-BASED SMART TQDM++
Training with LOS

Epoch 1/2 [Standard TQDM]: 100%|██████████| 88/88 [03:02<00:00,  2.08s/it, acc=56.57%, loss=0.832]


Epoch 1 completed: Loss=0.8320, Acc=56.57%


Epoch 2/2 [Standard TQDM]: 100%|██████████| 88/88 [02:51<00:00,  1.95s/it, acc=68.93%, loss=0.641]


Epoch 2 completed: Loss=0.6408, Acc=68.93%

All training demos completed!
Notice how SmartBar provides intelligent emoji feedback based on performance trends!
